Импортируем все необходимые библиотеки

In [184]:
pip install nltk natasha scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [198]:
import nltk

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer

import pymorphy2

from natasha import Doc, Segmenter, MorphVocab, NewsEmbedding, NewsMorphTagger

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

import pandas as pd

In [146]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Пользователь\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Пользователь\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Пользователь\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [148]:
stop_words = set(stopwords.words('russian'))
lemmatizer = WordNetLemmatizer()

In [150]:
texts = [
    "Мама мыла раму и пела песню.",
    "Сегодня прекрасная погода, и мы пошли гулять в парк.",
    "Студенты изучают программирование на Python и создают интересные проекты."
]

In [165]:
def lemm_and_stem(text):
    # Инициализация необходимых компонентов
    segmenter = Segmenter()
    Vocab = MorphVocab()
    Embeddings = NewsEmbedding()
    Tagger = NewsMorphTagger(Embeddings)
    Stemmer = SnowballStemmer("russian")
    morph = pymorphy2.MorphAnalyzer()

    # Токенизация и лемматизация
    doc = Doc(text)
    doc.segment(segmenter)
    doc.tag_morph(Tagger)
    
    lems = []
    for token in doc.tokens:
        # Игнорируем токены, которые являются знаками препинания
        if token.lemma and token.lemma.isalnum():  
            lems.append(token.lemma)
        else:
            parsed = morph.parse(token.text)[0]
            # Добавляем только те токены, которые являются буквами или цифрами
            if parsed.normal_form.isalnum():
                lems.append(parsed.normal_form)
    
    return lems

In [167]:
lemms = [lemm_and_stem(text) for text in texts]

In [175]:
def tokenize_text(text):
    tokens = word_tokenize(text.lower())
    tokens = [word for word in tokens if word.isalnum() and word not in stop_words]
    return tokens

In [177]:
tokens = [tokenize_text(text) for text in texts]

In [182]:
print("Результаты NLTK:")
print(tokens)

print("Результаты NLTK:")
print(lemms)

Результаты NLTK:
[['мама', 'мыла', 'раму', 'пела', 'песню'], ['сегодня', 'прекрасная', 'погода', 'пошли', 'гулять', 'парк'], ['студенты', 'изучают', 'программирование', 'python', 'создают', 'интересные', 'проекты']]
Результаты NLTK:
[['мама', 'мыло', 'рама', 'и', 'петь', 'песня'], ['сегодня', 'прекрасный', 'погода', 'и', 'мы', 'послать', 'гулять', 'в', 'парк'], ['студент', 'изучать', 'программирование', 'на', 'python', 'и', 'создавать', 'интересный', 'проект']]


In [188]:
def bag_of_words(texts):
    # Инициализация векторайзера для Bag of Words
    vectorizer = CountVectorizer()

    # Преобразуем тексты в вектор признаков
    X = vectorizer.fit_transform(texts)
    
    # Получаем список всех уникальных слов (фич)
    feature_names = vectorizer.get_feature_names_out()
    
    # Преобразуем результат в массив
    result = X.toarray()
    
    return feature_names, result

In [206]:
bow_features, bow_matrix = bag_of_words(texts)

bow_df = pd.DataFrame(bow_matrix, columns=bow_features)
bow_df.index = [f"Предложение {i+1}" for i in range(len(texts))]

In [208]:
def tf_idf(texts):
    # Инициализация векторайзера для TF-IDF
    vectorizer = TfidfVectorizer()

    # Преобразуем тексты в матрицу TF-IDF
    X = vectorizer.fit_transform(texts)
    
    # Получаем список всех уникальных слов (фич)
    feature_names = vectorizer.get_feature_names_out()
    
    # Преобразуем результат в массив
    result = X.toarray()
    
    return feature_names, result

In [210]:
tfidf_features, tfidf_matrix = tf_idf(texts)

# Выводим результат
tfidf_df = pd.DataFrame(tfidf_matrix, columns=tfidf_features)
tfidf_df.index = [f"Предложение {i+1}" for i in range(len(texts))]

In [212]:
print("Bag of Words:")
display(bow_df)

Bag of Words:


,python,гулять,изучают,интересные,мама,мы,мыла,на,парк,пела,песню,погода,пошли,прекрасная,программирование,проекты,раму,сегодня,создают,студенты
Предложение 1,0,0,0,0,1,0,1,0,0,1,1,0,0,0,0,0,1,0,0,0
Предложение 2,0,1,0,0,0,1,0,0,1,0,0,1,1,1,0,0,0,1,0,0
Предложение 3,1,0,1,1,0,0,0,1,0,0,0,0,0,0,1,1,0,0,1,1


In [214]:
print("\nTF-IDF:")
display(tfidf_df)


TF-IDF:


,python,гулять,изучают,интересные,мама,мы,мыла,на,парк,пела,песню,погода,пошли,прекрасная,программирование,проекты,раму,сегодня,создают,студенты
Предложение 1,0.000000,0.000000,0.000000,0.000000,0.447214,0.000000,0.447214,0.000000,0.000000,0.447214,0.447214,0.000000,0.000000,0.000000,0.000000,0.000000,0.447214,0.000000,0.000000,0.000000
Предложение 2,0.000000,0.377964,0.000000,0.000000,0.000000,0.377964,0.000000,0.000000,0.377964,0.000000,0.000000,0.377964,0.377964,0.377964,0.000000,0.000000,0.000000,0.377964,0.000000,0.000000
Предложение 3,0.353553,0.000000,0.353553,0.353553,0.000000,0.000000,0.000000,0.353553,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.353553,0.353553,0.000000,0.000000,0.353553,0.353553
